In [1]:
import pandas as pd

In [2]:
url = "https://docs.google.com/spreadsheets/d/1vFWwNJ7-61O_QYaPo8d7bmw7K192az4R/export"
data_excel = pd.read_excel(url, sheet_name=0, header=9)
data_excel = data_excel.iloc[:, :-11]

In [3]:
#Obtengo las columnas que me interesan
#data = data.iloc[:, -10:]
col_sexo = data_excel.iloc[:, 0]
col_baremos = data_excel.iloc[:, -11:]

data  = pd.concat( [col_sexo, col_baremos], axis=1 )
data.head()

,Sexo,Somatización,Obsesión/com,Sen. Emocio.,Depresión,Ansiedad,Hostilidad,Ans. Fóbica,Ideación Paran.,Psicoticismo,Agotamiento,SA45
0,1,Promedio,Alto,Bajo,Promedio,Promedio,Promedio,Bajo,Bajo,Promedio,Bajo,Bajo
1,1,Alto,Promedio,Bajo,Alto,Promedio,Bajo,Promedio,Alto,Bajo,Bajo,Bajo
2,1,Bajo,Promedio,Promedio,Bajo,Bajo,Promedio,Bajo,Alto,Bajo,Alto,Bajo
3,1,Alto,Alto,Alto,Alto,Alto,Alto,Promedio,Alto,Bajo,Alto,Promedio
4,1,Promedio,Bajo,Alto,Alto,Promedio,Bajo,Alto,Alto,Bajo,Bajo,Bajo


In [4]:
#Creo el map
category_map = {
    'Bajo': 0,
    'Promedio': 1,
    'Alto': 2
}

In [5]:
#Convierto las categorias a numeros
data2 = data.copy()
for i in range(11):
    data2.iloc[:, 1+i ] = data2.iloc[:, 1+i ].map(category_map)

data2.head()

,Sexo,Somatización,Obsesión/com,Sen. Emocio.,Depresión,Ansiedad,Hostilidad,Ans. Fóbica,Ideación Paran.,Psicoticismo,Agotamiento,SA45
0,1,1,2,0,1,1,1,0,0,1,0,0
1,1,2,1,0,2,1,0,1,2,0,0,0
2,1,0,1,1,0,0,1,0,2,0,2,0
3,1,2,2,2,2,2,2,1,2,0,2,1
4,1,1,0,2,2,1,0,2,2,0,0,0


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score

In [7]:
X = data2.drop('Agotamiento', axis=1).values
y = data2['Agotamiento'].values.astype(int)

In [8]:
# Dividir conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [9]:
#Normalizamos los datos

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
print(X_train.shape[1])

11


In [11]:
# Convertir datos a tensores de PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 11)#, 22)
        self.fc2 = nn.Linear(11, 6)
        self.fc3 = nn.Linear(6, 3)

        self.relu = nn.ReLU()
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
# Inicializar la red
model = MLP()

# Definir la función de pérdida y el optimizador
#criterion = nn.BCELoss()  # Pérdida de entropía cruzada binaria
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [14]:
epochs = 2000
patience = 20  # Número de épocas permitidas con F1 decreciente
best_f1 = 0.0
best_model_state = None

print_each = 10

# Entrenamiento
for epoch in range(epochs):
    model.train()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluación durante el entrenamiento
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
        f1 = f1_score(y_test.numpy(), predicted.numpy(), average='weighted')
    
    # Comparar con el mejor F1
    if f1 >= best_f1:
        best_f1 = f1
        best_model_state = model.state_dict()  # Guardar el mejor modelo
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, F1: {f1:.4f}')
        
    #if (epoch + 1) % print_each == 0:
    #    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, F1: {f1:.4f}')

    #if counter >= patience:
    #    print("Early stopping triggered.")
    #    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, F1: {f1:.4f}')
    #    break

Epoch 1/2000, Loss: 1.0575, F1: 0.2571
Epoch 2/2000, Loss: 1.0570, F1: 0.2571
Epoch 3/2000, Loss: 1.0564, F1: 0.2571
Epoch 4/2000, Loss: 1.0558, F1: 0.2571
Epoch 5/2000, Loss: 1.0552, F1: 0.2571
Epoch 6/2000, Loss: 1.0546, F1: 0.2571
Epoch 7/2000, Loss: 1.0540, F1: 0.2571
Epoch 8/2000, Loss: 1.0535, F1: 0.2571
Epoch 9/2000, Loss: 1.0529, F1: 0.2571
Epoch 10/2000, Loss: 1.0523, F1: 0.2571
Epoch 11/2000, Loss: 1.0517, F1: 0.2571
Epoch 12/2000, Loss: 1.0511, F1: 0.2571
Epoch 13/2000, Loss: 1.0505, F1: 0.2571
Epoch 14/2000, Loss: 1.0499, F1: 0.2571
Epoch 15/2000, Loss: 1.0493, F1: 0.2571
Epoch 16/2000, Loss: 1.0487, F1: 0.2571
Epoch 17/2000, Loss: 1.0481, F1: 0.2571
Epoch 18/2000, Loss: 1.0474, F1: 0.2571
Epoch 19/2000, Loss: 1.0468, F1: 0.2571
Epoch 20/2000, Loss: 1.0462, F1: 0.2571
Epoch 21/2000, Loss: 1.0455, F1: 0.2571
Epoch 22/2000, Loss: 1.0449, F1: 0.2571
Epoch 23/2000, Loss: 1.0443, F1: 0.2571
Epoch 24/2000, Loss: 1.0436, F1: 0.2571
Epoch 25/2000, Loss: 1.0430, F1: 0.2571
Epoch 26/